In [1]:
import numpy as np
import pandas as pd
from functools import reduce
import pickle

import pyspark
spark = pyspark.sql.SparkSession.builder.getOrCreate()

import plotly.express as px

In [2]:
with open('./data/closed_listings_cleaned.pickle', 'rb') as read_file:
    df = pickle.load(read_file)
df.head()

,Unnamed: 0,property_id,listing_id,status,list_date,list_price,sold_price,year_build,baths_3qtr,sold_date,...,num_beds,zipcode,state,lon,lat,city,address,county_flips_code,county,sold_price_pct_chg
0,0,1328063592,2929815961,sold,2021-05-28,769950.0,827500.0,1961.0,0.0,2021-06-30,...,4.0,91343,California,-118.481091,34.229127,North Hills,16012 Londelius St,6037.0,Los Angeles,7.474511
1,1,9865727012,2927867659,sold,2021-04-07,752990.0,730000.0,2021.0,0.0,2021-07-13,...,3.0,90650,California,-118.075178,33.911295,Norwalk,12044 Olive St,6037.0,Los Angeles,-3.053161
2,2,1449014840,2929777709,sold,2021-05-27,535000.0,579000.0,1988.0,0.0,2021-07-12,...,3.0,90650,California,-118.067705,33.918579,Norwalk,12415 Imperial Hwy Unit 47,6037.0,Los Angeles,8.224299
3,3,2009321584,2929560940,sold,2021-05-21,700000.0,740000.0,1949.0,0.0,2021-07-02,...,3.0,90240,California,-118.124144,33.957706,Downey,9548 Brock Ave,6037.0,Los Angeles,5.714286
4,4,1556603542,2929775353,sold,2021-05-27,860000.0,950000.0,1969.0,1.0,2021-06-25,...,4.0,90703,California,-118.076873,33.855240,Cerritos,11964 Agnes St,6037.0,Los Angeles,10.465116


In [3]:
sold_by_zip = df.groupby('city')[['list_price','sold_price']].median().reset_index()
sold_by_zip['sold_price_pct_chg_by_city'] = (sold_by_zip.sold_price - sold_by_zip.list_price)/sold_by_zip.list_price *100

In [4]:
df_tmp = df.merge(sold_by_zip, on = 'city')

In [22]:
df_tmp.rename(columns = {"list_price_y": "median_list_price_by_city",
               "sold_price_y" : "median_sold_price_by_city"}, inplace = True)

In [96]:
df_tmp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 73631 entries, 0 to 73630
Data columns (total 30 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Unnamed: 0                  73631 non-null  int64  
 1   property_id                 73631 non-null  int64  
 2   listing_id                  73631 non-null  int64  
 3   status                      73631 non-null  object 
 4   list_date                   73630 non-null  object 
 5   list_price_x                73627 non-null  float64
 6   sold_price_x                73631 non-null  float64
 7   year_build                  72665 non-null  float64
 8   baths_3qtr                  73631 non-null  float64
 9   sold_date                   73631 non-null  object 
 10  baths_full                  73631 non-null  float64
 11  baths_half                  73631 non-null  float64
 12  lot_sqft                    67348 non-null  float64
 13  sqrt                        736

#### Create datasets for APP use

In [75]:
with open('./data/map_data.pickle', 'wb') as to_write:
    pickle.dump(df_tmp, to_write)

In [120]:
df_tmp['sold_month'] = df_tmp['sold_date'].apply(lambda x: x[:7])

In [121]:
df_tmp.head()

,Unnamed: 0,property_id,listing_id,status,list_date,list_price_x,sold_price_x,year_build,baths_3qtr,sold_date,...,lat,city,address,county_flips_code,county,sold_price_pct_chg,median_list_price_by_city,median_sold_price_by_city,sold_price_pct_chg_by_city,sold_month
0,0,1328063592,2929815961,sold,2021-05-28,769950.0,827500.0,1961.0,0.0,2021-06-30,...,34.229127,North Hills,16012 Londelius St,6037.0,Los Angeles,7.474511,695000.0,755000.0,8.633094,2021-06
1,6186,2987266522,2930179667,sold,2021-06-05,395000.0,430000.0,1980.0,0.0,2021-07-09,...,34.225072,North Hills,8505 Burnet Ave Unit D,6037.0,Los Angeles,8.860759,695000.0,755000.0,8.633094,2021-07
2,6187,1307977249,2928971049,sold,2021-05-06,765000.0,785000.0,1953.0,0.0,2021-06-30,...,34.236502,North Hills,9131 Aqueduct Ave,6037.0,Los Angeles,2.614379,695000.0,755000.0,8.633094,2021-06
3,6949,1662935352,2929573768,sold,2021-05-21,699000.0,835000.0,1956.0,1.0,2021-06-18,...,34.232777,North Hills,8920 Hayvenhurst Ave,6037.0,Los Angeles,19.456366,695000.0,755000.0,8.633094,2021-06
4,9930,1657718120,2929558803,sold,2021-05-22,799000.0,885500.0,1956.0,2.0,2021-07-13,...,34.232680,North Hills,16624 Bahama St,6037.0,Los Angeles,10.826033,695000.0,755000.0,8.633094,2021-07


In [122]:
sold_by_neigborhood = df_tmp.groupby(['city','zipcode','sold_month'])['property_id'].count().reset_index()

In [123]:
sold_by_neigborhood.rename(columns = {'property_id':'sold'}, inplace = True)

In [124]:
sold_by_neigborhood.head()

,city,zipcode,sold_month,sold
0,Acampo,95220,2020-12,1
1,Acampo,95220,2021-01,3
2,Acampo,95220,2021-03,5
3,Acampo,95220,2021-04,11
4,Acampo,95220,2021-05,11


In [125]:
sold_by_neigborhood['zipcode'] = sold_by_neigborhood['zipcode'].astype(str)

In [126]:
with open('./data/sold_by_neig.pickle', 'wb') as to_write:
    pickle.dump(sold_by_neigborhood, to_write)

In [111]:
df_tmp[(df_tmp['city'] == 'Acampo')&(df_tmp['zipcode'] == 95220)&(df_tmp['address'] == '22900 N Sowles Rd')].to_csv('check.csv')

In [127]:
sold_by_type= df_tmp.groupby(['zipcode','property_type','sold_month'])['property_id'].count().reset_index()

In [129]:
sold_by_type.rename(columns = {'property_id':'sold'}, inplace = True)


In [132]:
sold_by_type['zipcode'] = sold_by_type['zipcode'].astype(str)

In [134]:
with open('./data/sold_by_type.pickle', 'wb') as to_write:
    pickle.dump(sold_by_type, to_write)